In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [4]:
file = pd.read_csv('data/train_1.csv', header=None)

In [5]:
file.head()

,0,1
0,big 5 검사 해봤다 너무 자다 맞다 놀라다 ㅋㅋㅋ esfp 같다 ? ㅎㅎ,1
1,"6/30 목요일 마음 둘곳 같이 가시다 분 계시다 ^-^( MBTI , 뇌파 검사 ...",1
2,E / I 만 다른 유형 의 판단 기능 은 어떤 식 으로 차이 가 나나 요 ? 예 ...,0
3,나 는 천천히 가다 사람 이다 . 나 는 천천히 가다 사람 이다 . 하지만 뒤 로는...,0
4,요즘 카페 글 을 보다 예민하다 글 들 이 .. 몇몇 글 들 을 보다 질문 자 가 ...,0


In [6]:
token = file[0]
label = file[1]

In [11]:
sentences = token.to_list()
label_list = label.to_list()

In [12]:
token_list = [s.split() for s in sentences]

In [17]:
import gensim
model = gensim.models.Word2Vec(sentences=token_list, size=300, window=5, workers=4, min_count=1)

In [18]:
# export the word embedding
model.wv.save_word2vec_format('mbti_word2vec.txt', binary=False)

In [19]:
import os

embeddings_index= {}
f = open(os.path.join('', 'mbti_word2vec.txt'), encoding = 'utf-8')
for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:])
    embeddings_index[word] = coefs
f.close()

71821it [00:09, 7945.95it/s]


In [20]:
# Set the max length

length = list()

for i in range(len(token_list)):
    length.append(len(token_list[i]))

max_length = max(length)

In [21]:
max_length

18011

In [23]:
VALIDATION_SPLIT = 0.2

# vectorize the text samples into a 2D integer tensor
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(token_list)
sequences = tokenizer_obj.texts_to_sequences(token_list)

In [24]:
# pad sequences
word_index = tokenizer_obj.word_index
print('Found %s unique tokens.' % len(word_index))

Found 69601 unique tokens.


In [25]:
blurb_pad = pad_sequences(sequences, maxlen=max_length)
print('Shape of blurb tensor:', blurb_pad.shape)

Shape of blurb tensor: (22855, 18011)


In [26]:
# split the data into a training set and a validation set
indices = np.arange(blurb_pad.shape[0])
np.random.shuffle(indices)
blurb_pad = blurb_pad[indices]
state = label[indices]
num_validation_samples = int(VALIDATION_SPLIT * blurb_pad.shape[0])

In [27]:
X_train_pad = blurb_pad[:-num_validation_samples]
y_train = state[:-num_validation_samples]
X_test_pad = blurb_pad[-num_validation_samples:]
y_test = state[-num_validation_samples:]

In [28]:
print('Shape of X_train_pad tensor:', X_train_pad.shape)
print('Shape of y_train tensor:', y_train.shape)

print('Shape of X_test_pad tensor:', X_test_pad.shape)
print('Shape of y_test tensor:', y_test.shape)

Shape of X_train_pad tensor: (18284, 18011)
Shape of y_train tensor: (18284,)
Shape of X_test_pad tensor: (4571, 18011)
Shape of y_test tensor: (4571,)


In [29]:
EMBEDDING_DIM =300
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in tqdm(word_index.items()):
    if i > num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

100%|██████████████████████████████████████████████████████████████████████████| 69601/69601 [00:24<00:00, 2865.98it/s]


In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, GRU
from tensorflow.keras.layers import Embedding
from tensorflow.keras.initializers import Constant

# define model
model = Sequential()
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=max_length,
                            trainable=False)
model.add(embedding_layer)
model.add(GRU(units=32,  dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print('Summary of the built model...')
print(model.summary())

Summary of the built model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 18011, 300)        20880600  
_________________________________________________________________
gru (GRU)                    (None, 32)                32064     
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 20,912,697
Trainable params: 32,097
Non-trainable params: 20,880,600
_________________________________________________________________
None


In [36]:
X_train_pad = np.asarray(X_train_pad)
y_train = np.asarray(y_train)
X_test_pad = np.asarray(X_test_pad)
y_test = np.asarray(y_test)

In [37]:
print('Train...')

model.fit(X_train_pad, y_train, batch_size=32, epochs=20, validation_data=(X_test_pad, y_test), verbose=2)

Train...
Train on 18284 samples, validate on 4571 samples
Epoch 1/20


KeyboardInterrupt: 